In [1]:
import pandas as pd
import sqlite3

### Шаг 1

(Балл - 2) Необходимо скачать CSV-файл - «transactions_for_dz2.csv», создать таблицу transaction_bd со всеми полями, загрузить данные из файла в таблицу и оставить таблицу со структурой:

(0, 'TRANSACTION_ID', 'INTEGER', 0, None, 0) - id транзакции
(1, 'TX_DATETIME', 'NUMERIC', 0, None, 0) - дата транзакции
(2, 'CUSTOMER_ID', 'INTEGER', 0, None, 0) - id клиента
(3, 'TERMINAL_ID', 'INTEGER', 0, None, 0) - id терминала
(4, 'TX_AMOUNT', 'REAL', 0, None, 0) - сумма транзакции

Также необходимо скачать второй CSV-файл - «client_info.csv», создать таблицу customer_bd со всеми полями, загрузить данные из файла в таблицу и получить таблицу со структурой:

(0, 'CLIENT_ID', 'INTEGER', 0, None, 0) - id клиента
(1, 'START_DT', 'NUMERIC', 0, None, 0) - дата начало записи о клиенте
(2, 'END_DT', 'NUMERIC', 0, None, 0) - дата закрытия записи о клиенте
(3, 'CLIENT_NAME', 'TEXT', 0, None, 0) - название клиента
(4, 'YEAR_BIRTH', 'TEXT', 0, None, 0) - дата рождение клиента

Примечание для следующих шагов: сумма транзакций не может быть null-ом, это всегда число.

In [2]:
transaction_df = pd.read_csv('./transactions_for_dz2.csv')
customer_df = pd.read_csv('./client_info.csv', sep=';')

In [3]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   TRANSACTION_ID  1048575 non-null  int64  
 1   TX_DATETIME     1048575 non-null  object 
 2   CUSTOMER_ID     1048575 non-null  int64  
 3   TERMINAL_ID     1048575 non-null  int64  
 4   TX_AMOUNT       1048575 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 40.0+ MB


In [4]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4988 entries, 0 to 4987
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   START_DT     4988 non-null   object
 1   END_DT       4988 non-null   object
 2   CLIENT_NAME  4988 non-null   object
 3   YEAR_BIRTH   4988 non-null   int64 
 4   CLIENT_ID    4988 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 195.0+ KB


In [5]:
#Создание таблиц
con = sqlite3.connect("./hw3.db")
sql_drop_transaction_bd = "DROP TABLE IF EXISTS transaction_bd;"
sql_drop_customer_bd = "DROP TABLE IF EXISTS customer_bd;"
sql_create_transaction_bd = """
CREATE TABLE transaction_bd (
    TRANSACTION_ID INTEGER NOT NULL
                           PRIMARY KEY,
    TX_DATETIME    NUMERIC NOT NULL,
    CUSTOMER_ID    INTEGER NOT NULL,
    TERMINAL_ID    INTEGER NOT NULL,
    TX_AMOUNT   REAL    NOT NULL
);
"""
sql_create_customer_bd = """
CREATE TABLE customer_bd (
    CLIENT_ID   INTEGER PRIMARY KEY
                        NOT NULL,
    START_DT    NUMERIC NOT NULL,
    END_DT      NUMERIC,
    CLIENT_NAME TEXT,
    YEAR_BIRTH  NUMERIC
);
"""
cur = con.cursor()
cur.execute(sql_drop_transaction_bd)
cur.execute(sql_create_transaction_bd)
cur.execute(sql_drop_customer_bd)
cur.execute(sql_create_customer_bd)
con.commit()

In [6]:
#Заполнение данными
sql_insert_transaction_bd = """
INSERT INTO transaction_bd (
                               TRANSACTION_ID,
                               TX_DATETIME,
                               CUSTOMER_ID,
                               TERMINAL_ID,
                               TX_AMOUNT
                           )
                           VALUES (
                               ?,
                               ?,
                               ?,
                               ?,
                               ?
                           );
"""
sql_insert_customer_bd = """
INSERT INTO customer_bd (
                            START_DT,
                            END_DT,
                            CLIENT_NAME,
                            YEAR_BIRTH,
                            CLIENT_ID
                        )
                        VALUES (
                            ?,
                            ?,
                            ?,
                            ?,
                            ?
                        );

"""
con.executemany(sql_insert_transaction_bd, transaction_df.values.tolist())
con.executemany(sql_insert_customer_bd, customer_df.values.tolist())
con.commit()


### Шаг 2

(Балл - 1 за каждый пункт) Написать скрипты:

In [7]:
# a. Найти имя клиента/клиентов с максимальной суммой транзакций за весь период
# (клиент должен быть действующим, то есть дата закрытия записи о клиенте
# не должна быть меньше дня относительно которого мы считаемся).
# За дефолтную дату для выборки действующих клиентов брать ‘2023-05-01’
sql_query_a = """
SELECT 
    CLIENT_NAME,
    MAX(SUM_TX_AMOUNT)
FROM
(
SELECT 
    tr.CUSTOMER_ID,
    cus.CLIENT_NAME,
    SUM(tr.TX_AMOUNT) as SUM_TX_AMOUNT
FROM transaction_bd as tr
    join customer_bd as cus on cus.CLIENT_ID = tr.CUSTOMER_ID
WHERE cus.END_DT >= '2023-05-01'
GROUP BY tr.CUSTOMER_ID, cus.CLIENT_NAME
)
"""
table_a = pd.read_sql(sql_query_a, con)
print(table_a)

  CLIENT_NAME  MAX(SUM_TX_AMOUNT)
0       Kylee           786115.87


In [8]:
# b. Найти имя клиента/клиентов с минимальной суммой транзакций за весь период
# (клиент должен быть действующим, то есть дата закрытия записи о клиенте
# не должна быть меньше дня относительно которого мы считаемся).
# За дефолтную дату для выборки действующих клиентов брать ‘2023-05-01’
sql_query_b = """
SELECT 
    CLIENT_NAME,
    MIN(SUM_TX_AMOUNT)
FROM
(
SELECT 
    tr.CUSTOMER_ID,
    cus.CLIENT_NAME,
    SUM(tr.TX_AMOUNT) as SUM_TX_AMOUNT
FROM transaction_bd as tr
    join customer_bd as cus on cus.CLIENT_ID = tr.CUSTOMER_ID
WHERE cus.END_DT >= '2023-05-01'
GROUP BY tr.CUSTOMER_ID, cus.CLIENT_NAME
)
"""
table_b = pd.read_sql(sql_query_b, con)
print(table_b)



  CLIENT_NAME  MIN(SUM_TX_AMOUNT)
0        Kloe               30.48


In [9]:
# c. Найти сумму транзакций относительно даты 2023-04-01 для клиентов, у которых id начинается с 111
# (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше
# дня относительно которого мы считаемся). Транзакции учитываются только после завершении дня.
# За дефолтную дату для выборки действующих клиентов брать ‘2023-05-01’
sql_query_c = """
SELECT 
       CUSTOMER_ID,
       SUM(TX_AMOUNT) AS SUM_TX_AMOUNT
FROM transaction_bd
WHERE
    TX_DATETIME < '2023-04-02'
    and CUSTOMER_ID in
    (
    SELECT CLIENT_ID
    FROM customer_bd
    WHERE substr(CLIENT_ID,0,4) = '111'
        and END_DT >= '2023-05-01'
    )  
GROUP BY CUSTOMER_ID
"""
table_c = pd.read_sql(sql_query_c, con)
print(table_c)

   CUSTOMER_ID  SUM_TX_AMOUNT
0         1112       35273.16
1         1113      230646.39
2         1114      113183.47
3         1115       38690.76
4         1116      196387.92
5         1117       52751.58
6         1118      401049.14
7         1119      223525.69


In [11]:
# d. Найти сумму транзакций относительно года рождения клиентов 
# (клиент должен быть действующим, то есть дата закрытия записи о клиенте
# не должна быть меньше дня относительно которого мы считаемся).
# Сортировать по убыванию года рождения.
sql_query_d = """
SELECT 
    cus.YEAR_BIRTH,
    round(SUM(tr.TX_AMOUNT),2) as SUM_TX_AMOUNT
FROM transaction_bd as tr
    join customer_bd as cus on cus.CLIENT_ID = tr.CUSTOMER_ID
WHERE cus.END_DT >= '2023-05-01'
GROUP BY cus.YEAR_BIRTH
ORDER BY cus.YEAR_BIRTH DESC"""
table_d = pd.read_sql(sql_query_d, con)
print(table_d)

    YEAR_BIRTH  SUM_TX_AMOUNT
0         2000    86327196.30
1         1999     1041506.56
2         1998      601675.01
3         1997     8479942.93
4         1996     1483252.51
5         1995     6321199.64
6         1994     6244280.80
7         1993     5732058.13
8         1992     5251433.30
9         1991     7141073.57
10        1990     6617599.94
11        1989     6303724.26
12        1988     7471929.98
13        1987     6629354.18
14        1986     5026130.51
15        1985     6742343.15
16        1984     7277775.24
17        1983      339072.02
18        1982    13709527.61
19        1981     5321969.97
20        1980     9232299.44
21        1979     5679293.41
22        1978     4994142.38
23        1977    10338469.52
24        1976    11666129.24
25        1975     6553700.61
26        1974    10173952.36
27        1973    10875457.40
28        1972    10649250.40
29        1971     7283031.89
30        1970    10012763.38
31        1969     6179549.55
32        

In [12]:
# e. Найти количество транзакций для каждого клиента (клиент должен быть действующим, 
# то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).
# Сортировать по убыванию количества транзакций.
sql_query_e = """
SELECT tr.CUSTOMER_ID,
       COUNT(tr.TRANSACTION_ID) AS CNT_TRANSACTION_ID
  FROM transaction_bd AS tr
       JOIN
       customer_bd AS cus ON cus.CLIENT_ID = tr.CUSTOMER_ID
 WHERE cus.END_DT >= '2023-05-01'
 GROUP BY tr.CUSTOMER_ID;"""
table_e = pd.read_sql(sql_query_e, con)
print(table_e)

      CUSTOMER_ID  CNT_TRANSACTION_ID
0               0                 201
1               1                 382
2               2                 202
3               5                 342
4               6                 427
...           ...                 ...
3731         4991                 155
3732         4992                 357
3733         4993                 196
3734         4996                 263
3735         4997                 230

[3736 rows x 2 columns]


### Шаг 3

(Балл - 3) Найти сумму транзакций за каждый месяц (январь, февраль, март, апрель) для всех клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [16]:
sql_query_step3 = """
SELECT 
    CASE 
        WHEN STRFTIME('%m', tr.TX_DATETIME) = '01' THEN 'январь'
        WHEN STRFTIME('%m', tr.TX_DATETIME) = '02' THEN 'ферваль'
        WHEN STRFTIME('%m', tr.TX_DATETIME) = '03' THEN 'март'
        WHEN STRFTIME('%m', tr.TX_DATETIME) = '04' THEN 'апрель'
        ELSE 'ой' END AS MONTH,
       SUM(tr.TX_AMOUNT) AS SUM_TX_AMOUNT
  FROM transaction_bd AS tr
       JOIN
       customer_bd AS cus ON cus.CLIENT_ID = tr.CUSTOMER_ID
 WHERE cus.END_DT >= '2023-05-01'
 GROUP BY STRFTIME('%m', tr.TX_DATETIME);"""
table_step3 = pd.read_sql(sql_query_step3, con)
print(table_step3)

     MONTH  SUM_TX_AMOUNT
0   январь   1.159199e+08
1  ферваль   1.062295e+08
2     март   1.150534e+08
3   апрель   7.261812e+07
